MY NN

In [1]:
import os
import cv2
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from tensorflow import keras
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from matplotlib import cm
from PIL import Image, ImageOps
from keras import backend as K
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, InputLayer, MaxPooling2D, AveragePooling2D
from tensorflow.python.keras.layers import Convolution2D
from tensorflow.python.keras import layers, models, Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip ./drive/MyDrive/data/mrdata.zip

Archive:  ./drive/MyDrive/data/mrdata.zip
   creating: mrdata/
   creating: mrdata/10/
   creating: mrdata/10/depth/
  inflating: mrdata/10/depth/11.29.24.441.png  
  inflating: mrdata/10/depth/11.29.24.554.png  
  inflating: mrdata/10/depth/11.29.24.663.png  
  inflating: mrdata/10/depth/11.29.24.772.png  
  inflating: mrdata/10/depth/11.29.24.883.png  
  inflating: mrdata/10/depth/11.29.24.997.png  
  inflating: mrdata/10/depth/11.29.25.104.png  
  inflating: mrdata/10/depth/11.29.25.210.png  
  inflating: mrdata/10/depth/11.29.25.322.png  
  inflating: mrdata/10/depth/11.29.25.433.png  
  inflating: mrdata/10/depth/11.29.25.543.png  
  inflating: mrdata/10/depth/11.29.25.651.png  
  inflating: mrdata/10/depth/11.29.25.764.png  
  inflating: mrdata/10/depth/mirror11.29.24.405.png  
  inflating: mrdata/10/depth/mirror11.29.24.489.png  
  inflating: mrdata/10/depth/mirror11.29.24.565.png  
  inflating: mrdata/10/depth/mirror11.29.24.646.png  
  inflating: mrdata/10/depth/mirror11.29.24

In [4]:
W = 512
H = 424
max_value_for_normalization = -1

def make_df(path_to_xlsx, path_to_folder):
    if not os.path.exists(path_to_xlsx):
        print('[Err] Fail xlsx path')
        return
    if not os.path.exists(path_to_folder):
        print('[Err] Fail folder path')
        return

    df = pd.read_excel(path_to_xlsx)
    df = df.fillna(0)
    df['Body length (cm)'] = df['Body length (cm)'].astype(int)
    df['Chest girth (cm)'] = df['Chest girth (cm)'].astype(int)
    df['Withers height (cm)'] = df['Withers height (cm)'].astype(int)
    df['Chest depth (cm)'] = df['Chest depth (cm)'].astype(int)
    df['Chest width (cm)'] = df['Chest width (cm)'].astype(int)
    df['Live weight (kg)'] = df['Live weight (kg)'].astype(int)
    df.drop(
        columns=['Frame', 'Content', 'Number of pig', 'Left', 'Right', 'Quality', 'Transform\nT_right_to_left', 'Sex',
                 'Age (days)'], inplace=True)

    df = df.iloc[2:]
    df['image'] = None
    for index, row in df.iterrows():
        path = f'{path_to_folder}/{index}/depth'
        if not os.path.exists(path):
            continue
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        print(f'For index {index} count {len(files)} files')
        for file in files:
            raw_image = cv2.imread(f'{path}/{file}', cv2.CV_16U).astype(np.float32)
            raw_image = cv2.resize(raw_image, (W, H))
            raw_image = raw_image.reshape(-1, W, H, 1)
            global max_value_for_normalization
            if np.amax(raw_image) > max_value_for_normalization:
                max_value_for_normalization = np.amax(raw_image)

            new_data = {
                'image': raw_image,
                'Live weight (kg)': df.iloc[index - 3]['Live weight (kg)'],
                'Body length (cm)': df.iloc[index - 3]['Body length (cm)'],
                'Chest girth (cm)': df.iloc[index - 3]['Chest girth (cm)'],
                'Withers height (cm)': df.iloc[index - 3]['Withers height (cm)'],
                'Chest depth (cm)': df.iloc[index - 3]['Chest depth (cm)'],
                'Chest width (cm)': df.iloc[index - 3]['Chest width (cm)'],
            }

            df = pd.concat([df, pd.DataFrame.from_records([new_data])])

    df['image'] = df['image'] / max_value_for_normalization
    df = df.iloc[16:]
    df = df.reset_index().drop(columns=['index'])
    return df

dataframe = make_df('./drive/MyDrive/data/data.xlsx', 'mrdata')
dataframe

For index 3 count 31 files
For index 4 count 32 files
For index 5 count 21 files
For index 6 count 31 files
For index 7 count 26 files
For index 8 count 39 files
For index 9 count 27 files
For index 10 count 27 files
For index 11 count 24 files
For index 12 count 27 files
For index 13 count 17 files
For index 14 count 37 files
For index 15 count 22 files
For index 16 count 33 files
For index 17 count 22 files


,Live weight (kg),Body length (cm),Chest girth (cm),Withers height (cm),Chest depth (cm),Chest width (cm),image
0,117,114,117,55,37,32,"[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
1,117,114,117,55,37,32,"[[[[0.], [0.285125], [0.], [0.], [0.13575], [0..."
2,117,114,117,55,37,32,"[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
3,117,114,117,55,37,32,"[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
4,117,114,117,55,37,32,"[[[[0.], [0.935375], [0.], [0.], [0.], [0.], [..."
...,...,...,...,...,...,...,...
411,118,123,117,63,37,32,"[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
412,118,123,117,63,37,32,"[[[[0.], [0.], [0.], [0.112625], [0.], [0.969]..."
413,118,123,117,63,37,32,"[[[[0.], [0.], [0.], [0.], [0.], [0.], [0.], [..."
414,118,123,117,63,37,32,"[[[[0.], [0.], [0.], [0.], [0.499625], [0.], [..."


In [5]:
# 'adam' 'rmsprop'
def create_model(opt, dense=6):
    model = Sequential()

    model.add(Convolution2D(64, (3, 3), activation='relu', input_shape=(W, H, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(dense, activation='relu'))
    
    model.compile(optimizer=opt, loss='mse', metrics=['mae', 'mape'])

    return model

In [6]:
columns_features_all = ['Live weight (kg)', 'Body length (cm)', 'Chest girth (cm)', 'Withers height (cm)', 'Chest depth (cm)', 'Chest width (cm)']
columns_features_kg = ['Live weight (kg)']
columns_features_sm = ['Body length (cm)', 'Chest girth (cm)', 'Withers height (cm)', 'Chest depth (cm)', 'Chest width (cm)']
# column_img = ['image']

ADAM ALL PARAMS

In [7]:
!rm -r './model/best'

count = 1
evals_models = []
ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'

  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=3) # 2 - 3
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_all].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 1)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_all].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 1)

  print("Create model")
  model = create_model(opt='adam')

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)


  !cp -av ./model/best '/content/drive/MyDrive/data/models/my_nn/best_adam_all_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)

  print("Write evaluate")
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 381.74228, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 381.74228 to 201.20915, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss improved from 201.20915 to 79.54464, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss did not improve from 79.54464

Epoch 00005: val_loss did not improve from 79.54464

Epoch 00006: val_loss did not improve from 79.54464
Epoch 00006: early stopping
'./model/best' -> '/content/drive/MyDrive/data/models/my_nn/best_adam_all_params_1'
'./model/best/variables' -> '/content/drive/MyDrive/data/models/my_nn/best_adam_all_params_1/variables'
'./model/best/variables/variables.index' -> '/content/drive/MyDrive/data/models/my_nn/best_adam_all_para

In [8]:
evals_models

[[70.44513702392578, 6.5349812507629395, 17.040313720703125],
 [74.41060638427734, 6.690526962280273, 18.034311294555664],
 [63.21235656738281, 6.431484222412109, 15.270872116088867],
 [70.81752014160156, 6.720175743103027, 17.213443756103516],
 [31.904577255249023, 4.282796859741211, 11.612723350524902],
 [41.09101867675781, 4.773187637329102, 17.649696350097656],
 [27.793222427368164, 3.664501428604126, 11.66164493560791],
 [31.858463287353516, 3.8460402488708496, 13.79882526397705]]

RMSPROP ALL PARAMS

In [9]:
!rm -r './model/best'

count = 1
evals_models = []
ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'

  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=3) # 2 - 3
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_all].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 1)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_all].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 1)

  print("Create model")
  model = create_model(opt='rmsprop')

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/my_nn/best_rmsprop_all_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)

  print("Write evaluate")
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 3139.04834, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 3139.04834 to 2200.26562, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss improved from 2200.26562 to 2181.43237, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss did not improve from 2181.43237

Epoch 00005: val_loss improved from 2181.43237 to 2176.89404, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00006: val_loss improved from 2176.89404 to 999.85785, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00007: val_loss improved from 999.85785 to 173.82614, saving model to ./model/best
INFO:tensorflow:Assets written to: 

In [11]:
evals_models

[[30.134859085083008, 3.8982901573181152, 11.72211742401123],
 [31.73341941833496, 4.014143943786621, 12.568795204162598],
 [26.427837371826172, 3.8185935020446777, 11.232207298278809],
 [29.665870666503906, 3.807215452194214, 14.351411819458008],
 [311.278564453125, 15.333681106567383, 33.747379302978516],
 [314.4024353027344, 15.301429748535156, 38.09716796875],
 [250.91883850097656, 11.7634859085083, 31.012245178222656],
 [261.4476318359375, 12.303854942321777, 33.69062805175781]]

ADAM KG PARAM

In [12]:
!rm -r './model/best'

count = 1
evals_models = []
ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'

  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=3) # 2 - 3
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_kg].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 1)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_kg].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 1)

  print("Create model")
  model = create_model(opt='adam', dense=1)

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/my_nn/best_adam_kg_param_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)

  print("Write evaluate")
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 179.36162, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss did not improve from 179.36162

Epoch 00003: val_loss improved from 179.36162 to 34.48577, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss did not improve from 34.48577

Epoch 00005: val_loss did not improve from 34.48577

Epoch 00006: val_loss did not improve from 34.48577
Epoch 00006: early stopping
'./model/best' -> '/content/drive/MyDrive/data/models/my_nn/best_adam_kg_param_1'
'./model/best/variables' -> '/content/drive/MyDrive/data/models/my_nn/best_adam_kg_param_1/variables'
'./model/best/variables/variables.index' -> '/content/drive/MyDrive/data/models/my_nn/best_adam_kg_param_1/variables/variables.index'
'./model/best/variables/variables.data-00000-of-00001' -> '/conte

In [13]:
evals_models

[[40.71671676635742, 5.572873592376709, 5.214914321899414],
 [40.49875259399414, 5.517154693603516, 5.118470191955566],
 [124.1679916381836, 9.175553321838379, 8.351649284362793],
 [167.0754852294922, 10.927352905273438, 9.924180030822754],
 [59.28791427612305, 6.471853733062744, 6.157095432281494],
 [63.72107696533203, 6.810477256774902, 6.4431328773498535],
 [41.53560256958008, 5.315334796905518, 4.918078899383545],
 [43.13134765625, 5.680400848388672, 5.266970157623291]]

RMSPROP KG PARAM

In [14]:
!rm -r './model/best'

count = 1
evals_models = []
ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'

  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=3) # 2 - 3
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_kg].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 1)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_kg].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 1)

  print("Create model")
  model = create_model(opt='rmsprop', dense=1)

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/my_nn/best_rmsprop_kg_param_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)

  print("Write evaluate")
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 801.70795, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss did not improve from 801.70795

Epoch 00003: val_loss improved from 801.70795 to 207.16156, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss did not improve from 207.16156

Epoch 00005: val_loss improved from 207.16156 to 143.48026, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00006: val_loss improved from 143.48026 to 42.51854, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00007: val_loss did not improve from 42.51854

Epoch 00008: val_loss did not improve from 42.51854

Epoch 00009: val_loss did not improve from 42.51854
Epoch 00009: early stopping
'./model/best' -> '/content/drive/MyDri

In [15]:
evals_models

[[44.553279876708984, 5.648974895477295, 5.226968288421631],
 [42.30814743041992, 5.394101142883301, 4.942973613739014],
 [65.10926818847656, 6.679625511169434, 6.343894004821777],
 [56.18593215942383, 6.376646995544434, 5.967752933502197],
 [50.8377685546875, 5.713244438171387, 5.343411445617676],
 [62.20857620239258, 6.575150966644287, 6.1278557777404785],
 [590.3818359375, 22.332656860351562, 21.101194381713867],
 [566.4503784179688, 22.099103927612305, 20.90279197692871]]

ADAM SM PARAMS

In [16]:
!rm -r './model/best'

count = 1
evals_models = []
ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'

  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=3) # 2 - 3
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_sm].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 1)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_sm].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 1)

  print("Create model")
  model = create_model(opt='adam', dense=5)

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/my_nn/best_adam_sm_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)

  print("Write evaluate")
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 536.76385, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 536.76385 to 348.82965, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss improved from 348.82965 to 171.52600, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss did not improve from 171.52600

Epoch 00005: val_loss improved from 171.52600 to 99.51131, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00006: val_loss did not improve from 99.51131

Epoch 00007: val_loss improved from 99.51131 to 37.13491, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00008: val_loss improved from 37.13491 to 32.29793, saving model to ./model/b

In [17]:
evals_models

[[25.855257034301758, 3.6776633262634277, 14.190244674682617],
 [28.03647232055664, 3.908700942993164, 15.264863967895508],
 [38.31354904174805, 4.650640964508057, 14.906315803527832],
 [41.77621078491211, 4.727860450744629, 18.834766387939453],
 [33.374725341796875, 3.945712089538574, 13.301669120788574],
 [45.58894348144531, 4.290767192840576, 21.582839965820312],
 [62.324989318847656, 6.454316139221191, 16.224376678466797],
 [64.32828521728516, 6.465334892272949, 18.2232723236084]]

RMSPROP SM PARAMS

In [18]:
!rm -r './model/best'

count = 1
evals_models = []
ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):
  path = './model/best'

  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=3) # 2 - 3
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_sm].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 1)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_sm].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 1)

  print("Create model")
  model = create_model(opt='rmsprop', dense=5)

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=6, verbose=0)

  !cp -av ./model/best '/content/drive/MyDrive/data/models/my_nn/best_rmsprop_sm_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)

  print("Write evaluate")
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 727.63574, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss did not improve from 727.63574

Epoch 00003: val_loss improved from 727.63574 to 96.78851, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss did not improve from 96.78851

Epoch 00005: val_loss improved from 96.78851 to 90.30973, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00006: val_loss improved from 90.30973 to 69.86930, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00007: val_loss did not improve from 69.86930

Epoch 00008: val_loss did not improve from 69.86930

Epoch 00009: val_loss did not improve from 69.86930
Epoch 00009: early stopping
'./model/best' -> '/content/drive/MyDrive/da

In [19]:
evals_models

[[62.38288879394531, 5.942138195037842, 15.869012832641602],
 [68.12857055664062, 6.195609092712402, 17.054061889648438],
 [35.285667419433594, 4.272373199462891, 15.07740306854248],
 [47.11393356323242, 4.940667152404785, 19.874130249023438],
 [379.8737487792969, 18.50619888305664, 37.28992462158203],
 [381.37554931640625, 18.557144165039062, 40.823219299316406],
 [27.92353057861328, 4.013710021972656, 13.85926342010498],
 [31.783023834228516, 4.190462112426758, 16.123842239379883]]